In [3]:
from __future__ import division, print_function, absolute_import, unicode_literals
from builtins import str

import os

import pandas as pd
from matplotlib import pyplot as plt

DATA_PATH = os.path.abspath(os.path.join('..', '..', '..', 'Data'))

import string

In [4]:
import matplotlib
from IPython.display import display, HTML 
%matplotlib inline
np = pd.np
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

In [5]:
df = pd.read_csv(os.path.join(DATA_PATH, 'deduped_tweets.csv.gz'), low_memory=False)

In [6]:
df = df.drop_duplicates('id', keep='last')[['id', 'id_str', 'text']]
df.id == df.id_str
(df.id != df.id_str).sum()

0

In [7]:
df = df[['id', 'text']]

In [8]:
df.text

0         #python never stop learning what you enjoy doi...
1         Watching Boa vs. Python — https://t.co/Pivpk02s2A
2         Monty Python - The silly walk https://t.co/C0J...
                                ...                        
183067    RT @RealPython: List of Python API Wrappers &g...
183068    Watching Boa vs. Python — https://t.co/5THbrirfQO
183069    Чертова дюжина вакансий в IT и Digital /  / 1....
Name: text, dtype: object

In [9]:
df['tokens'] = df.text.str.split()
df

,id,text,tokens
0,724276510626979840,#python never stop learning what you enjoy doi...,"[#python, never, stop, learning, what, you, en..."
1,724276498249572352,Watching Boa vs. Python — https://t.co/Pivpk02s2A,"[Watching, Boa, vs., Python, —, https://t.co/P..."
2,724276388325412866,Monty Python - The silly walk https://t.co/C0J...,"[Monty, Python, -, The, silly, walk, https://t..."
...,...,...,...
183067,724275609858392066,RT @RealPython: List of Python API Wrappers &g...,"[RT, @RealPython:, List, of, Python, API, Wrap..."
183068,724275578879111169,Watching Boa vs. Python — https://t.co/5THbrirfQO,"[Watching, Boa, vs., Python, —, https://t.co/5..."
183069,724275568871673857,Чертова дюжина вакансий в IT и Digital / / 1....,"[Чертова, дюжина, вакансий, в, IT, и, Digital,..."


In [15]:
import collections
import re
tld_iana = pd.read_csv(os.path.join(DATA_PATH, 'tlds-from-iana.csv'))
tld_iana = collections.OrderedDict(sorted(zip((tld.strip().lstrip('.') for tld in tld_iana.domain),
                                              [(sponsor.strip(), -1) for sponsor in tld_iana.sponsor]),
                                          key=lambda x: len(x[0]),
                                          reverse=True))

# top 20 in Google searches per day
# sorted by longest first so .com matches before .om (Oman)
tld_popular = collections.OrderedDict(sorted([
    ('com', ('Commercial', 4860000000)),
    ('org', ('Noncommercial', 1950000000)),
    ('edu', ('US accredited postsecondary institutions', 1550000000)),
    ('gov', ('United States Government', 1060000000)),
    ('uk',  ('United Kingdom', 473000000)),
    ('net', ('Network services', 206000000)),
    ('ca', ('Canada', 165000000)),
    ('de', ('Germany', 145000000)),
    ('jp', ('Japan', 139000000)),
    ('fr', ('France', 96700000)),
    ('au', ('Australia', 91000000)),
    ('us', ('United States', 68300000)),
    ('ru', ('Russian Federation', 67900000)),
    ('ch', ('Switzerland', 62100000)),
    ('it', ('Italy', 55200000)),
    ('nl', ('Netherlands', 45700000)),
    ('se', ('Sweden', 39000000)),
    ('no', ('Norway', 32300000)),
    ('es', ('Spain', 31000000)),
    ('mil', ('US Military', 28400000)),
    ], key=lambda x: len(x[0]), reverse=True))
uri_schemes_iana = sorted(pd.read_csv(os.path.join(DATA_PATH, 'uri-schemes.xhtml.csv'),
                                      index_col=0).index.values,
                          key=lambda x: len(str(x)), reverse=True)
uri_schemes_popular = ['chrome-extension', 'example', 'content', 'bitcoin',
                       'telnet', 'mailto',
                       'https', 'gtalk',
                       'http', 'smtp', 'feed',
                       'udp', 'ftp', 'ssh', 'git', 'apt', 'svn', 'cvs']
# email = re.compile(r'^([\w-]+(?:\.[\w-]+)*)@((?:[\w-]+\.)*\w[\w-]{0,66})\.([a-z]{2,6}(?:\.[a-z]{2})?)')
fqdn = r'(\b[a-zA-Z0-9-.]+([.]' + r'|'.join(tld_iana) + r')\b)'
fqdn_popular = r'(\b[a-zA-Z0-9-.]+\b([.]' + r'|'.join(tld_popular) + r'\b)\b)'
username = r'(\b[a-zA-Z0-9-.!#$%&*+-/=?^_`{|}~]+\b)'

email = re.compile(r'(\b' + username + r'\b@\b' + fqdn + r'\b)')
email_popular = re.compile(r'(\b' + username + r'\b@\b' + fqdn_popular + r'\b)')

# TODO: unmatched surrounding symbols are accepted/consumed, likewise for multiple dots/ats
at = r'(([-@="_(\[{\|\s]+(at|At|AT)[-@="_)\]\}\|\s]+)|[@])'
dot = r'(([-.="_(\[{\|\s]+(dot|dt|Dot|DOT)[-.="_)\]\}\|\s]+)|[.])'
fqdn_obfuscated = r'(\b(([a-zA-Z0-9-]+' + dot + r'){1,7})(' + r'|'.join(tld_iana) + r')\b)'
fqdn_popular_obfuscated = r'(\b(([a-zA-Z0-9-]+' + dot + r'){1,7})(' + r'|'.join(tld_popular) + r')\b)'
username_obfuscated = r'(([a-zA-Z0-9!#$%&*+/?^`~]+' + dot + r'?){1,7})'
email_obfuscated = re.compile(r'(\b' + username_obfuscated + at + fqdn_obfuscated + r'\b)')
email_popular_obfuscated = re.compile(r'(\b' + username_obfuscated + at + fqdn_popular_obfuscated + r'\b)')

url_path = r'(\b[^\s]+)'
url_scheme = r'(\b(' + '|'.join(uri_schemes_iana) + r')[:][/]{2})'
url_scheme_popular = r'(\b(' + '|'.join(uri_schemes_popular) + r')[:][/]{2})'
url = r'(\b' + url_scheme + fqdn + url_path + r'?\b)'

In [16]:
df['tokens'] = df.text.str.replace(url, '').str.split()
df

,id,text,tokens
0,724276510626979840,#python never stop learning what you enjoy doi...,"[#python, never, stop, learning, what, you, en..."
1,724276498249572352,Watching Boa vs. Python — https://t.co/Pivpk02s2A,"[Watching, Boa, vs., Python, —]"
2,724276388325412866,Monty Python - The silly walk https://t.co/C0J...,"[Monty, Python, -, The, silly, walk, via, @You..."
...,...,...,...
183067,724275609858392066,RT @RealPython: List of Python API Wrappers &g...,"[RT, @RealPython:, List, of, Python, API, Wrap..."
183068,724275578879111169,Watching Boa vs. Python — https://t.co/5THbrirfQO,"[Watching, Boa, vs., Python, —]"
183069,724275568871673857,Чертова дюжина вакансий в IT и Digital / / 1....,"[Чертова, дюжина, вакансий, в, IT, и, Digital,..."


In [17]:
df['txt'] = df.text.str.replace(url, ' ').str.replace(r'\W+', ' ').str.replace(r'\s+', ' ')
df.txt

0          python never stop learning what you enjoy doing 
1                                   Watching Boa vs Python 
2                   Monty Python The silly walk via YouTube
                                ...                        
183067    RT RealPython List of Python API Wrappers gt g...
183068                              Watching Boa vs Python 
183069    Чертова дюжина вакансий в IT и Digital 1 Go ра...
Name: txt, dtype: object

In [18]:
df['txt'] = df.txt.str.replace(r'\d+', ' ').str.replace(r'\s+', ' ')
df['tokens'] = df.txt.str.split()
df

,id,text,tokens,txt
0,724276510626979840,#python never stop learning what you enjoy doi...,"[python, never, stop, learning, what, you, enj...",python never stop learning what you enjoy doing
1,724276498249572352,Watching Boa vs. Python — https://t.co/Pivpk02s2A,"[Watching, Boa, vs, Python]",Watching Boa vs Python
2,724276388325412866,Monty Python - The silly walk https://t.co/C0J...,"[Monty, Python, The, silly, walk, via, YouTube]",Monty Python The silly walk via YouTube
...,...,...,...,...
183067,724275609858392066,RT @RealPython: List of Python API Wrappers &g...,"[RT, RealPython, List, of, Python, API, Wrappe...",RT RealPython List of Python API Wrappers gt g...
183068,724275578879111169,Watching Boa vs. Python — https://t.co/5THbrirfQO,"[Watching, Boa, vs, Python]",Watching Boa vs Python
183069,724275568871673857,Чертова дюжина вакансий в IT и Digital / / 1....,"[Чертова, дюжина, вакансий, в, IT, и, Digital,...",Чертова дюжина вакансий в IT и Digital Go разр...


Notice that we trounced the hashtag #Python  
That's not good.  
Can you fix it?  
Anything else we might be messing up?  
*what other punctuation marks have special meaning in Tweets*  

In [19]:
# improve on the "stopword" filters here
#
# :-) (ask me about a smilie lexicon)
# not-so-simple words? (ask me about a regex for compound words)
# python variables names with underscores? (regex)

In [26]:
f = os.path.join(DATA_PATH, 'text.csv.gz')
df.to_csv(f, encoding='utf8', compression='gzip', quoting=pd.io.common.csv.QUOTE_NONNUMERIC)

In [27]:
import gzip
with gzip.open(os.path.join(DATA_PATH, 'text.csv.gz'), 'rb') as f:
    df = pd.read_csv(f)

Make sure you can read it back in!

In [28]:
df = pd.DataFrame.from_csv(os.path.join(DATA_PATH, 'text.csv.gz'))
df

,id,text,tokens,txt
0,724276510626979840,#python never stop learning what you enjoy doi...,"['python', 'never', 'stop', 'learning', 'what'...",python never stop learning what you enjoy doing
1,724276498249572352,Watching Boa vs. Python — https://t.co/Pivpk02s2A,"['Watching', 'Boa', 'vs', 'Python']",Watching Boa vs Python
2,724276388325412866,Monty Python - The silly walk https://t.co/C0J...,"['Monty', 'Python', 'The', 'silly', 'walk', 'v...",Monty Python The silly walk via YouTube
...,...,...,...,...
183067,724275609858392066,RT @RealPython: List of Python API Wrappers &g...,"['RT', 'RealPython', 'List', 'of', 'Python', '...",RT RealPython List of Python API Wrappers gt g...
183068,724275578879111169,Watching Boa vs. Python — https://t.co/5THbrirfQO,"['Watching', 'Boa', 'vs', 'Python']",Watching Boa vs Python
183069,724275568871673857,Чертова дюжина вакансий в IT и Digital / / 1....,"['Чертова', 'дюжина', 'вакансий', 'в', 'IT', '...",Чертова дюжина вакансий в IT и Digital Go разр...
